In [1]:
from pandas import DataFrame
import numpy as np

In [2]:
train_data = DataFrame.from_csv('data/train.csv')
y = train_data.index
train_data.index = range(len(train_data))

In [3]:
train = train_data.sample(frac=0.9)
validate = train_data[~train_data.index.isin(train.index)]

In [4]:
print(len(train))
print(len(validate))
print(len(train) + len(validate))
print(len(train_data))

37800
4200
42000
42000


In [7]:
from keras.models import Sequential

Using TensorFlow backend.


In [8]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y[train.index])
y_validate = to_categorical(y[validate.index])
print(len(y_train))
print(len(y_validate))

37800
4200


In [9]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(min_delta=1e-5, patience=10)

In [10]:
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l1, l2, l1_l2, L1L2
from keras.constraints import max_norm

In [23]:
model = None
model = Sequential()
model.add(
    Dense(300, activation='relu', input_dim=(28 * 28), kernel_initializer='TruncatedNormal',
))
#model.add(Dropout(0.05))
model.add(
    Dense(300, activation='relu', kernel_initializer='TruncatedNormal',
    kernel_regularizer=l2(0.01),
    #kernel_constraint=max_norm(2.)
))
model.add(Dropout(0.3))
model.add(
    Dense(300, activation='relu', kernel_initializer='TruncatedNormal',
    #kernel_regularizer=l2(0.01),
    #kernel_constraint=max_norm(2.)
))
model.add(Dropout(0.3))
model.add(
    Dense(300, activation='relu', kernel_initializer='TruncatedNormal',
    #kernel_regularizer=l2(0.01),
    #kernel_constraint=max_norm(2.)
))
model.add(Dropout(0.3))
model.add(
    Dense(300, activation='relu', kernel_initializer='TruncatedNormal',
    #kernel_regularizer=l2(0.01),
    #kernel_constraint=max_norm(2.)
))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

In [24]:
from keras.optimizers import RMSprop
model.summary()
model.compile(
    optimizer="adam",#RMSprop(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_8 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_5 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_6 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_7 (Dropout)          (None, 300)               0         
__________

In [25]:
history = model.fit(
    train.as_matrix() / 255.0,
    y_train,
    validation_data=(validate.as_matrix() / 255.0, y_validate), 
    epochs=500,
    batch_size=128,
    verbose=2,
    callbacks=[es]
)

Train on 37800 samples, validate on 4200 samples
Epoch 1/500
5s - loss: 0.8061 - acc: 0.8476 - val_loss: 0.2705 - val_acc: 0.9471
Epoch 2/500
4s - loss: 0.2403 - acc: 0.9534 - val_loss: 0.2726 - val_acc: 0.9467
Epoch 3/500
4s - loss: 0.1899 - acc: 0.9647 - val_loss: 0.2014 - val_acc: 0.9640
Epoch 4/500
4s - loss: 0.1646 - acc: 0.9702 - val_loss: 0.2017 - val_acc: 0.9638
Epoch 5/500
4s - loss: 0.1443 - acc: 0.9762 - val_loss: 0.2102 - val_acc: 0.9629
Epoch 6/500
4s - loss: 0.1300 - acc: 0.9790 - val_loss: 0.1914 - val_acc: 0.9664
Epoch 7/500
4s - loss: 0.1260 - acc: 0.9796 - val_loss: 0.2134 - val_acc: 0.9640
Epoch 8/500
4s - loss: 0.1191 - acc: 0.9813 - val_loss: 0.2069 - val_acc: 0.9650
Epoch 9/500
4s - loss: 0.1073 - acc: 0.9839 - val_loss: 0.1885 - val_acc: 0.9688
Epoch 10/500
4s - loss: 0.1020 - acc: 0.9857 - val_loss: 0.2169 - val_acc: 0.9648
Epoch 11/500
4s - loss: 0.0983 - acc: 0.9860 - val_loss: 0.2280 - val_acc: 0.9643
Epoch 12/500
4s - loss: 0.0938 - acc: 0.9868 - val_loss: 0

# Test model on validation sample
This should yield the same score as the val_acc in the output of the training above

In [26]:
val_predictions = model.predict_classes(validate.as_matrix() / 255.0, verbose=0)

In [27]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y[validate.index], val_predictions))

0.970952380952


# Prepare submission

In [28]:
target_data = DataFrame.from_csv('data/test.csv', index_col=None)

In [29]:
predictions = model.predict_classes(target_data.as_matrix() / 255.0, verbose=0)

In [30]:
print(predictions)

[2 0 9 ..., 3 9 2]


In [31]:
np.savetxt('digit_prediction.csv', 
           np.c_[range(1,len(target_data)+1),predictions], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt='%d')